In [26]:
import pandas as pd
import numpy as np
import networkx as nx
import scipy.sparse as sp
from scipy.sparse import lil_matrix, csr_matrix

Since I don't need `log_index`, `transaction_hash` and `block_number` for my analysis I will not load them.

In [2]:
columns = ['token_address', 'from_address', 'to_address', 'value']
transactions_df = pd.read_csv('./datasets/ethereum-exchanges/token_transfers_full.csv')[columns]

Let’s check how many rows and columns we imported.

In [3]:
transactions_df.shape

(38901039, 4)

I can also check the names of the columns in the DataFrame.

In [4]:
transactions_df.columns

Index(['token_address', 'from_address', 'to_address', 'value'], dtype='object')

Let's have a look at the first 5 rows of my DataFrame

In [5]:
transactions_df.head(5)

,token_address,from_address,to_address,value
0,0xb64ef51c888972c908cfacf59b47c1afbc0ab8ac,0x78903c52de3923d203881f1b766160cba5a37f59,0xc20d6d7d2e7cde1cabc7f20c553fe93dada380f3,209000000
1,0xb64ef51c888972c908cfacf59b47c1afbc0ab8ac,0x78903c52de3923d203881f1b766160cba5a37f59,0xc4c49dbf1b4bc997c9a9758c1b1f86d33da4232d,191000000
2,0xb64ef51c888972c908cfacf59b47c1afbc0ab8ac,0x78903c52de3923d203881f1b766160cba5a37f59,0xab194b0c3e3423ec6cbe44fdf096cb43251d1ccd,181000000
3,0xb64ef51c888972c908cfacf59b47c1afbc0ab8ac,0x78903c52de3923d203881f1b766160cba5a37f59,0xe29b47e4d0cc68b847226c652f2a5e73d1a94343,204000000
4,0xb64ef51c888972c908cfacf59b47c1afbc0ab8ac,0x78903c52de3923d203881f1b766160cba5a37f59,0x5d0a1a7c7507142f06c04ffd604ab7b85b442f98,209000000


Let's take a look of the Data Types in my DataFrame

In [6]:
transactions_df.dtypes

token_address    object
from_address     object
to_address       object
value            object
dtype: object

Next, let’s see how much memory our DataFrame is using broken down by column.

In [7]:
transactions_df.memory_usage(deep=True)

Index                   132
token_address    3851202861
from_address     3851202861
to_address       3851202861
value            2947317077
dtype: int64

In [8]:
transactions_df.sort_values(['value'], ascending=[1])

,token_address,from_address,to_address,value
4216296,0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c,0x31aca4f978c148a8fd200852ea50f9dbfaf60c11,0x8d0035249a9c06d548550c0902b3496201caf7e4,0
25566660,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,0x8c80cbc68d4189b4f1b419c541ffe4caa9502b1e,0x4fb2eaab169720fcef1948eb926d28fb0a850e58,0
7030814,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xeeca53c3a421b2e292700ddaef54ef34759ac6fd,0x5239cdc0b3b7b34a65d89ebe6fe9ae075f216260,0
25566646,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,0x8c80cbc68d4189b4f1b419c541ffe4caa9502b1e,0x4fb2eaab169720fcef1948eb926d28fb0a850e58,0
25566633,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,0x8c80cbc68d4189b4f1b419c541ffe4caa9502b1e,0x4fb2eaab169720fcef1948eb926d28fb0a850e58,0
...,...,...,...,...
32171764,0x6b175474e89094c44da98b954eedeac495271d0f,0x493c57c4763932315a328269e1adad09653b9081,0x2f3437c6bf500dd985c154ed01ef6507811d68f4,99999999999999999999999
18776820,0x8e1b448ec7adfc7fa35fc2e885678bd323176e34,0x11790ee0357e84dfdf12d472e5c1ac8f17026b21,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,999999999999999999999999
29981496,0x8e870d67f660d95d5be530380d0ec0bd388289e1,0xfecd350addf607b2a6596aedde744e087748bcca,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,999999999999999999999999
12935237,0x8e1b448ec7adfc7fa35fc2e885678bd323176e34,0x229e86b4c4292904f78bfb0a4190be36a06565ed,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,999999999999999999999999


In [9]:
transactions_df.groupby(['value']).size()

value
0                           106667
1                            12858
10                            3256
100                            824
1000                          5055
                             ...  
9999999999999999999997           4
9999999999999999999998           5
9999999999999999999999         128
99999999999999999999999         57
999999999999999999999999         4
Length: 13867521, dtype: int64

Since I'm dealing with a huge dataset of almpst 39.000.000 rows I will discard transaction that are less of 10.000.000.000.000 Wei that corresponds to 0.00001 Ether [see the conversion rate](https://etherscan.io/unitconverter). In order to do so I nee my values in 'value' to be all numeric. This will be also needed in order to create my directed weighted graph.

In [11]:
transactions_df = transactions_df.astype({'value': 'float'})
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38901039 entries, 0 to 38901038
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   token_address  object 
 1   from_address   object 
 2   to_address     object 
 3   value          float64
dtypes: float64(1), object(3)
memory usage: 1.2+ GB


In [15]:
transactions_df.drop(transactions_df[transactions_df['value'] <= 10000000000000].index, inplace=True)
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35356454 entries, 19 to 38901038
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   token_address  object 
 1   from_address   object 
 2   to_address     object 
 3   value          float64
dtypes: float64(1), object(3)
memory usage: 1.3+ GB


In [20]:
# Calculate the number of unique senders and receivers
unique_senders = transactions_df['from_address'].nunique()
unique_receivers = transactions_df['to_address'].nunique()

print(f"Number of unique senders: {unique_senders}")
print(f"Number of unique receivers: {unique_receivers}")


Number of unique senders: 2875584
Number of unique receivers: 5124159


In [27]:
%%time

# Create a list of unique addresses
unique_addresses = np.unique(np.concatenate((transactions_df['from_address'], transactions_df['to_address'])))

# Create a dictionary for address-to-index mapping
address_to_index = {address: index for index, address in enumerate(unique_addresses)}

CPU times: user 2min 30s, sys: 1.83 s, total: 2min 32s
Wall time: 2min 32s


In [28]:
%%time

# Initialize the sparse matrix
num_addresses = len(unique_addresses)
sparse_matrix = csr_matrix((num_addresses, num_addresses), dtype=np.float64)

In [29]:
%%time

# Populate the sparse matrix with transaction values
for _, row in transactions_df.iterrows():
    from_index = address_to_index[row['from_address']]
    to_index = address_to_index[row['to_address']]
    sparse_matrix[from_index, to_index] += row['value']
    
scipy.sparse.save_npz('sparse_matrix.npz', sparse_matrix)

KeyboardInterrupt: 

In [ ]:
%%time

# Convert the sparse matrix to a NetworkX MultiDiGraph
G = nx.MultiDiGraph()

# Add nodes and edges from the sparse matrix
for from_index, to_index in zip(*sparse_matrix.nonzero()):
    from_address = unique_addresses[from_index]
    to_address = unique_addresses[to_index]
    transaction_value = sparse_matrix[from_index, to_index]

    G.add_node(from_address)
    G.add_node(to_address)
    G.add_edge(from_address, to_address, value=transaction_value)

# Print basic graph information
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Now you have a NetworkX MultiDiGraph (G) representing your transactions.

In [ ]:
#Graph Info
print(nx.info(G))

#Check if graph is Directed - True/False
print(nx.is_directed(G))

#Count of Nodes
print(nx.number_of_nodes(G))

#Count of Edges
print(nx.number_of_edges(G))